# IGTI - Cientista de Dados - Módulo 2
## Desafio - Parte 1

In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\spark-3.1.2-bin-hadoop2.7'

In [2]:
import pyspark
from pyspark import SparkContext
sc = SparkContext()

In [3]:
from pyspark.sql import SparkSession

In [4]:
#from pyspark.sql.functions import rand, randn
from pyspark.sql import SQLContext

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
spark = SparkSession.builder.appName("Desafio").getOrCreate()

In [7]:
# importa com inferencia dos tipos das colunas
df = spark.read.options(header='True', inferSchema='True').csv("stroke_data.csv")

### Perguntas

**1) Quantos registros existem no arquivo?**

In [8]:
df.count()

67135

**2) Quantas colunas existem no arquivo? Quantas são numéricas? Ao ler o arquivo com spark.read.csv, habilite inferSchema=True. Use a função printSchema() da API de Dataframes.**

In [9]:
df.printSchema()
# Resposta: 12 e 7

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



**3) (pergunta questionada no fórum de dúvidas e alterada pelo prof. em mensagem no fórum de avisos)**  

**Original:**  
> 3) No conjunto de dados, quantos pacientes estavam doentes?
> * 200 e 200
> * 165 e 138
> * 155 e 191
> * 136 e 166

**Nova:**
> 3) No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke), respectivamente?
> * 40287 e 26848
> * 40200 e 26841
> * 26848 e 40287
> * 26841 e 40287

In [10]:
df.show()

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

In [11]:
df.groupBy(df.stroke).count().show()
# Resposta: 40287 e 26848

+------+-----+
|stroke|count|
+------+-----+
|     1|40287|
|     0|26848|
+------+-----+



**4) A partir do dataframe, crie uma tabela temporária usando df.createOrReplaceTempView('table') e, a seguir, use spark.sql para escrever uma consulta SQL que obtenha quantos pacientes tiveram derrame por tipo de trabalho (work_type). Quantos pacientes que sofreram derrame trabalhavam, respectivamente, no setor privado, de forma independente, no governo e quantas são crianças?**

In [12]:
tb = df.createOrReplaceTempView('table')

spark.sql('select work_type, count(*) qtd from table where stroke = 1 group by work_type order by qtd desc').show()

# Resposta: 23711, 10807, 5164 e 520

+-------------+-----+
|    work_type|  qtd|
+-------------+-----+
|      Private|23711|
|Self-employed|10807|
|     Govt_job| 5164|
|     children|  520|
| Never_worked|   85|
+-------------+-----+



**5) Escreva uma consulta com spark.sql para determinar a proporção, por gênero, de participantes do estudo. A maioria dos participantes é:**

In [8]:
spark.sql('select gender, count(*) qtd from table group by gender order by count(*) desc').show()

# Resposta: Feminina

+------+-----+
|gender|  qtd|
+------+-----+
|Female|39530|
|  Male|27594|
| Other|   11|
+------+-----+



**6) Escreva uma consulta com spark.sql para determinar quem tem mais probabilidade de sofrer derrame: hipertensos ou não-hipertensos. Você pode escrever uma consulta para cada grupo. A partir das probabilidades que você obteve, você conclui que:**

In [9]:
print('With hypertension:')
spark.sql('select stroke, count(*) qde, (count(*) / (select count(*) from table where hypertension = 1) * 100) perc from table where hypertension = 1 group by stroke order by stroke').show()

print('Without hypertension:')
spark.sql('select stroke, count(*) qde, (count(*) / (select count(*) from table where hypertension = 0) * 100) perc from table where hypertension = 0 group by stroke order by stroke').show()

# A distribuição do percentual entre quem não tem hipertensão é mais equilibrado: 44% e 56%.
# Já entre os que tem hipertensão, a maioria dos que sofreram derrame tinham hipertensão (80%).

# Resposta: A hipertensão, neste conjunto de dados, aumenta a probabilidade de derrame.

With hypertension:
+------+----+------------------+
|stroke| qde|              perc|
+------+----+------------------+
|     0|2200|19.969138603975676|
|     1|8817| 80.03086139602432|
+------+----+------------------+

Without hypertension:
+------+-----+-----------------+
|stroke|  qde|             perc|
+------+-----+-----------------+
|     0|24648|43.92173634128087|
|     1|31470|56.07826365871913|
+------+-----+-----------------+



**7) Escreva uma consulta com spark.sql que determine o número de pessoas que sofreram derrame por idade. Com qual idade o maior número de pessoas do conjunto de dados sofreu derrame?**

In [10]:
spark.sql('select age, count(*) qtd from table group by age order by qtd desc').show(1)

# Resposta: 79

+----+----+
| age| qtd|
+----+----+
|79.0|3258|
+----+----+
only showing top 1 row



**8) Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos.**

In [11]:
#df.filter(df.age > 50).filter(df.stroke == 1).count()

df.where('age > 50 and stroke = 1').count()

# Resposta: 28938

28938

**9) Usando spark.sql, determine qual o nível médio de glicose para pessoas que, respectivamente, sofreram e não sofreram derrame.**

In [12]:
spark.sql('select stroke, avg(avg_glucose_level) from table group by stroke').show()

# Resposta: 119 e 103

+------+----------------------+
|stroke|avg(avg_glucose_level)|
+------+----------------------+
|     1|    119.95307046938272|
|     0|    103.60273130214506|
+------+----------------------+



**10) Qual é o BMI (IMC = índice de massa corpórea) médio de quem sofreu e não sofreu derrame?**

**NOTA:**  

A pergunta foi questionada no fórum de dúvidas pois as alterantivas apresentam alta precisão, com duas casas decimais, e os resultados encontrados não são iguais aos das alternativas, podendo inclusive se encaixar em situação onde os dois valores encontrados se enocntrem mais próximos de alternativas distintas.  
&nbsp;  
O Prof. instruiu, por mensagem no fórum de avisos, para que diferenças nas casas decimais fossem ignoradas.  

In [13]:
spark.sql('select stroke, avg(bmi) from table group by stroke').show()

df.groupBy('stroke').avg('bmi').show()

# Resposta: 29,81 e 27,86 (ignorando as casas decimais, conforme instrução)

+------+------------------+
|stroke|          avg(bmi)|
+------+------------------+
|     1|29.942490629729495|
|     0|27.989678933253657|
+------+------------------+

+------+------------------+
|stroke|          avg(bmi)|
+------+------------------+
|     1|29.942490629729495|
|     0|27.989678933253657|
+------+------------------+

